# Receber dados dos sensores com o protocolo MQTT.

In [2]:
import random
from paho.mqtt import client as mqtt_client


broker = 'broker.mqtt-dashboard.com'
port = 1883
client_id = f'python-mqtt-{random.randint(0, 100)}'
topicDistanceSensor = "4361fd9a-0c1e-420c-b858-27ad478288e6/distanceSensor"
topicAccelerometer = "4361fd9a-0c1e-420c-b858-27ad478288e6/accelerometer"
topicGyroscope = "4361fd9a-0c1e-420c-b858-27ad478288e6/gyroscope"
distancesData = []
accelerationsData = []
gyrosData = []

In [3]:
def handleMessage(msg):
    data = msg.payload.decode();
    topic = msg.topic
    if topic == topicDistanceSensor:
        handleDistanceValues(data)
    elif topic == topicAccelerometer:
        handleAccelerationValues(data)
    elif topic == topicGyroscope:
        handleGyroValues(data)


In [4]:
def handleDistanceValues(data):
    if data.find('|') != -1:
        parts = data.replace(' ', '').split('|')
        distances = []
        for element in parts:
            if element.find(':') != -1:
                value = int(element.split(':')[1])
                distances.append(value)
        
        print(distances)
        distancesData.append(distances)

In [5]:
def handleAccelerationValues(data):
    if data.find(':') != -1:
        acceleration = float(data.split(':')[1])
        print(acceleration)
        accelerationsData.append(acceleration)

In [6]:
def handleGyroValues(data):
    if data.find(':') != -1:
        gyro = float(data.split(':')[1])
        print(gyro)
        gyrosData.append(gyro)
        

In [7]:
def connectMqtt():
    def on_connect(client, userdata, flags, rc):
        if rc == 0:
            print("Conectado com sucesso ao Broker MQTT!")
        else:
            print("Falha ao se conectar, código de retorno %d\n", rc)

    client = mqtt_client.Client(client_id)
    client.on_connect = on_connect
    client.connect(broker, port)
    return client


def subscribe(client: mqtt_client):
    def on_message(client, userdata, msg):
        print(f"Recebido `{msg.payload.decode()}` do tópico `{msg.topic}`")
        handleMessage(msg)


    client.subscribe(topicDistanceSensor)
    client.subscribe(topicAccelerometer)
    client.subscribe(topicGyroscope)
    client.on_message = on_message

def runMQTT():
    client = connectMqtt()
    subscribe(client)
    client.loop_forever()

In [8]:
runMQTT()

Conectado com sucesso ao Broker MQTT!
Recebido `1:10 | 2:30 | 3:50` do tópico `4361fd9a-0c1e-420c-b858-27ad478288e6/distanceSensor`
[10, 30, 50]
Recebido `1:30 | 2:50 | 3:150` do tópico `4361fd9a-0c1e-420c-b858-27ad478288e6/distanceSensor`
[30, 50, 150]
Recebido `1:30 | 2:50 | 3:150` do tópico `4361fd9a-0c1e-420c-b858-27ad478288e6/distanceSensor`
[30, 50, 150]
Recebido `ax: 10 | ay: 20` do tópico `4361fd9a-0c1e-420c-b858-27ad478288e6/accelerometer`
[10.0, 20.0]
Recebido `gz:0.78` do tópico `4361fd9a-0c1e-420c-b858-27ad478288e6/gyroscope`
0.78


KeyboardInterrupt: 